# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 22 2022 12:09PM,247065,16,SHL-8560192,"SHL Pharma, LLC",Released
1,Sep 22 2022 12:06PM,247064,19,60016935,"GUSA Granules USA, Inc.",Released
2,Sep 22 2022 11:56AM,247062,19,60017897,"GUSA Granules USA, Inc.",Released
3,Sep 22 2022 11:53AM,247061,10,8559682,"Akron BioProducts, LLC",Released
4,Sep 22 2022 11:53AM,247060,10,EYE20092022,Eye Pharma Inc,Released
5,Sep 22 2022 11:46AM,247058,21,628371,"NBTY Global, Inc.",Released
6,Sep 22 2022 11:28AM,247056,10,SO94389,"Senseonics, Incorporated",Released
7,Sep 22 2022 11:28AM,247056,10,SO94390,"Senseonics, Incorporated",Released
8,Sep 22 2022 11:28AM,247056,10,SO94391,"Senseonics, Incorporated",Released
9,Sep 22 2022 11:28AM,247056,10,SO94392,"Senseonics, Incorporated",Executing


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
27,247060,Released,1
28,247061,Released,1
29,247062,Released,1
30,247064,Released,1
31,247065,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
247060,NaN,1.0
247061,NaN,1.0
247062,NaN,1.0
247064,NaN,1.0
247065,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
246890,0.0,1.0
246953,1.0,0.0
246977,0.0,13.0
246985,1.0,0.0
246989,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
246890,0,1
246953,1,0
246977,0,13
246985,1,0
246989,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,246890,0,1
1,246953,1,0
2,246977,0,13
3,246985,1,0
4,246989,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,246890,,1
1,246953,1,
2,246977,,13
3,246985,1,
4,246989,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 22 2022 12:09PM,247065,16,"SHL Pharma, LLC"
1,Sep 22 2022 12:06PM,247064,19,"GUSA Granules USA, Inc."
2,Sep 22 2022 11:56AM,247062,19,"GUSA Granules USA, Inc."
3,Sep 22 2022 11:53AM,247061,10,"Akron BioProducts, LLC"
4,Sep 22 2022 11:53AM,247060,10,Eye Pharma Inc
5,Sep 22 2022 11:46AM,247058,21,"NBTY Global, Inc."
6,Sep 22 2022 11:28AM,247056,10,"Senseonics, Incorporated"
10,Sep 22 2022 11:09AM,247048,20,"ACI Healthcare USA, Inc."
45,Sep 22 2022 11:02AM,247052,10,ISDIN Corporation
52,Sep 22 2022 10:56AM,247047,20,Alumier Labs Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Sep 22 2022 12:09PM,247065,16,"SHL Pharma, LLC",,1
1,Sep 22 2022 12:06PM,247064,19,"GUSA Granules USA, Inc.",,1
2,Sep 22 2022 11:56AM,247062,19,"GUSA Granules USA, Inc.",,1
3,Sep 22 2022 11:53AM,247061,10,"Akron BioProducts, LLC",,1
4,Sep 22 2022 11:53AM,247060,10,Eye Pharma Inc,,1
5,Sep 22 2022 11:46AM,247058,21,"NBTY Global, Inc.",,1
6,Sep 22 2022 11:28AM,247056,10,"Senseonics, Incorporated",1,3
7,Sep 22 2022 11:09AM,247048,20,"ACI Healthcare USA, Inc.",1,34
8,Sep 22 2022 11:02AM,247052,10,ISDIN Corporation,,7
9,Sep 22 2022 10:56AM,247047,20,Alumier Labs Inc.,,5


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 22 2022 12:09PM,247065,16,"SHL Pharma, LLC",1,
1,Sep 22 2022 12:06PM,247064,19,"GUSA Granules USA, Inc.",1,
2,Sep 22 2022 11:56AM,247062,19,"GUSA Granules USA, Inc.",1,
3,Sep 22 2022 11:53AM,247061,10,"Akron BioProducts, LLC",1,
4,Sep 22 2022 11:53AM,247060,10,Eye Pharma Inc,1,
5,Sep 22 2022 11:46AM,247058,21,"NBTY Global, Inc.",1,
6,Sep 22 2022 11:28AM,247056,10,"Senseonics, Incorporated",3,1
7,Sep 22 2022 11:09AM,247048,20,"ACI Healthcare USA, Inc.",34,1
8,Sep 22 2022 11:02AM,247052,10,ISDIN Corporation,7,
9,Sep 22 2022 10:56AM,247047,20,Alumier Labs Inc.,5,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 22 2022 12:09PM,247065,16,"SHL Pharma, LLC",1,
1,Sep 22 2022 12:06PM,247064,19,"GUSA Granules USA, Inc.",1,
2,Sep 22 2022 11:56AM,247062,19,"GUSA Granules USA, Inc.",1,
3,Sep 22 2022 11:53AM,247061,10,"Akron BioProducts, LLC",1,
4,Sep 22 2022 11:53AM,247060,10,Eye Pharma Inc,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Sep 22 2022 12:09PM,247065,16,"SHL Pharma, LLC",1.0,NaN
1,Sep 22 2022 12:06PM,247064,19,"GUSA Granules USA, Inc.",1.0,NaN
2,Sep 22 2022 11:56AM,247062,19,"GUSA Granules USA, Inc.",1.0,NaN
3,Sep 22 2022 11:53AM,247061,10,"Akron BioProducts, LLC",1.0,NaN
4,Sep 22 2022 11:53AM,247060,10,Eye Pharma Inc,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 22 2022 12:09PM,247065,16,"SHL Pharma, LLC",1.0,0.0
1,Sep 22 2022 12:06PM,247064,19,"GUSA Granules USA, Inc.",1.0,0.0
2,Sep 22 2022 11:56AM,247062,19,"GUSA Granules USA, Inc.",1.0,0.0
3,Sep 22 2022 11:53AM,247061,10,"Akron BioProducts, LLC",1.0,0.0
4,Sep 22 2022 11:53AM,247060,10,Eye Pharma Inc,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2470319,39.0,3.0
12,247040,1.0,0.0
15,494060,5.0,37.0
16,494107,5.0,0.0
19,988118,9.0,1.0
20,988161,41.0,2.0
21,987922,2.0,2.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2470319,39.0,3.0
1,12,247040,1.0,0.0
2,15,494060,5.0,37.0
3,16,494107,5.0,0.0
4,19,988118,9.0,1.0
5,20,988161,41.0,2.0
6,21,987922,2.0,2.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,39.0,3.0
1,12,1.0,0.0
2,15,5.0,37.0
3,16,5.0,0.0
4,19,9.0,1.0
5,20,41.0,2.0
6,21,2.0,2.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,39.0
1,12,Released,1.0
2,15,Released,5.0
3,16,Released,5.0
4,19,Released,9.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Executing,3.0,0.0,37.0,0.0,1.0,2.0,2.0
Released,39.0,1.0,5.0,5.0,9.0,41.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Executing,3.0,0.0,37.0,0.0,1.0,2.0,2.0
1,Released,39.0,1.0,5.0,5.0,9.0,41.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Executing,3.0,0.0,37.0,0.0,1.0,2.0,2.0
1,Released,39.0,1.0,5.0,5.0,9.0,41.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()